In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import stats
from pathlib import Path

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_map.figure_utils import depth_selectivity, closed_loop_rsof, get_session_list, depth_decoder
from v1_depth_map.figure_utils import common_utils as v1_common_utils
from tqdm import tqdm

In [ ]:
READ_VERSION = 9
READ_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{READ_VERSION}"
)

VERSION = 9
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"fig2").mkdir(parents=True, exist_ok=True)

reload = False # set to True to reload everything

In [ ]:
# LOAD DATA
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Concatenate all sessions for closedloop RS-OF integration data
models = ["gof", "grs", "gadd", "g2d", "gratio"]
cols = [
    "roi",
    "best_depth",
    "preferred_depth_closedloop",
    "preferred_depth_closedloop_crossval",
    "depth_tuning_popt_closedloop",
    "depth_tuning_test_rsq_closedloop",
    "depth_tuning_test_spearmanr_pval_closedloop",
    "depth_tuning_test_spearmanr_rval_closedloop",
    "preferred_RS_closedloop_g2d",
    "preferred_RS_closedloop_crossval_g2d",
    "preferred_OF_closedloop_g2d",
    "preferred_OF_closedloop_crossval_g2d",
]
cols_to_add = [
    "rsof_test_rsq_closedloop_",
    "rsof_rsq_closedloop_",
    "rsof_popt_closedloop_",
]
for model in models:
    for col in cols_to_add:
        cols.append(f"{col}{model}")

mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"])]
if reload:
    results_all = v1_common_utils.concatenate_all_neurons_df(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
            trialnum_min=10,
            mouse_list=mouse_list,
        ),
        filename="neurons_df.pickle",
        cols=cols,
        read_iscell=True,
        verbose=True,
    )
    results_all["preferred_RS_closedloop_crossval_g2d"] = (
        results_all["preferred_RS_closedloop_crossval_g2d"] * 100
    )
    results_all["preferred_depth_closedloop_crossval"] = (
        results_all["preferred_depth_closedloop_crossval"] * 100
    )
    results_all["preferred_OF_closedloop_crossval_g2d"] = np.degrees(
        results_all["preferred_OF_closedloop_crossval_g2d"]
    )
    results_all.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_closedloop.pickle")
else:
    results_all = pd.read_pickle(READ_ROOT / "fig2" /  "results_all_rsof_closedloop.pickle")

In [ ]:
# Load example session for open loop
project = "hey2_3d-vision_foodres_20220101"
session_name_openloop_trace = "PZAH6.4b_S20220524"
session_name_openloop = "PZAG3.4f_S20220523"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example_openloop_trace, trials_df_example_openloop_trace = spheres.sync_all_recordings(
    session_name=session_name_openloop_trace,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

vs_df_example_openloop, trials_df_example_openloop = spheres.sync_all_recordings(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example_openloop = pipeline_utils.create_neurons_ds(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example_openloop = pd.read_pickle(neurons_ds_example_openloop.path_full)

# Concatenate all sessions for openloop
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"
                ])]
if reload:
    results_all_openloop = v1_common_utils.concatenate_all_neurons_df(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=True,
            mouse_list=mouse_list,
            v1_only=True,
            trialnum_min=10,
        ),
        filename="neurons_df.pickle",
        cols=[
            "roi",
            "best_depth",
            "preferred_depth_closedloop",
            "preferred_depth_closedloop_crossval",
            "depth_tuning_popt_closedloop",
            "depth_tuning_test_rsq_closedloop",
            "depth_tuning_test_spearmanr_pval_closedloop",
            "depth_tuning_test_spearmanr_rval_closedloop",
            "preferred_RS_closedloop_g2d",
            "preferred_RS_closedloop_crossval_g2d",
            "preferred_OF_closedloop_g2d",
            "preferred_OF_closedloop_crossval_g2d",
            "rsof_rsq_closedloop_crossval_g2d",
            "rsof_rsq_closedloop_g2d",
            "preferred_RS_openloop_actual_g2d",
            "preferred_OF_openloop_actual_g2d",
            "rsof_rsq_openloop_actual_g2d",
            "rsof_popt_closedloop_g2d",
            "rsof_popt_openloop_actual_g2d",
            "rsof_test_rsq_closedloop_g2d",
            "rsof_test_rsq_openloop_actual_g2d",
        ],
        read_iscell=True,
        verbose=True,
    )
    results_all_openloop["preferred_RS_closedloop_g2d"] = (
        results_all_openloop["preferred_RS_closedloop_g2d"] * 100
    )
    results_all_openloop["preferred_depth_closedloop"] = (
        results_all_openloop["preferred_depth_closedloop"] * 100
    )
    results_all_openloop["preferred_OF_closedloop_g2d"] = np.degrees(
        results_all_openloop["preferred_OF_closedloop_g2d"]
    )
    results_all_openloop["preferred_RS_openloop_actual_g2d"] = (
        results_all_openloop["preferred_RS_openloop_actual_g2d"] * 100
    )
    results_all_openloop["preferred_OF_openloop_actual_g2d"] = np.degrees(
        results_all_openloop["preferred_OF_openloop_actual_g2d"]
    )

    results_all_openloop.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_openloop.pickle")
else:
    results_all_openloop = pd.read_pickle(READ_ROOT / "fig2" / "results_all_rsof_openloop.pickle")
print(f"openloop sessions {len(results_all_openloop.session.unique())}")
print(f"5 depth sessions {np.sum(pd.Series(results_all_openloop.session.unique()).str.contains('PZAH6.4b|PZAG3.4f'))}")

In [ ]:
# # concatenate all open loop depth tunings
# depth_tuning_openloop = v1_common_utils.concatenate_all_neurons_df(
#     flexilims_session=flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         exclude_openloop=False,
#         exclude_pure_closedloop=True,
#         mouse_list=mouse_list,
#         v1_only=True,
#         trialnum_min=10,
#     ),
#     filename="neurons_df_openloop.pickle",
#     read_iscell=True,
#     verbose=True,
# )

# cols = [i for i in depth_tuning_openloop.columns if i not in results_all_openloop.columns]
# results_all_openloop = pd.concat([results_all_openloop, depth_tuning_openloop[cols]], axis=1)
# results_all_openloop.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_openloop.pickle")

In [ ]:
# Load decoder results
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
if reload:
    decoder_results = depth_decoder.concatenate_all_decoder_results(
        flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=True,
            v1_only=True,
            trialnum_min=10,
        ),
        filename="decoder_results.pickle",
    )
    decoder_results.to_pickle(SAVE_ROOT / "fig2" / "decoder_results_openloop.pickle")
else:
    decoder_results = pd.read_pickle(READ_ROOT /  "fig2" / "decoder_results_openloop.pickle")

In [ ]:
# 1 example neuron, binned by RS, OF, and RS-OF
EXAMPLE_ROIS = [
    88,
]
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
model_labels = ["Optic flow", "Running speed", "Additive", "Conjunctive", "RS/OF"]
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]) + x[-1])

fig = plt.figure(figsize=(18 / 2.54, 21 / 2.54))
for iroi, roi in enumerate(EXAMPLE_ROIS):
    fig.add_axes([0.06, 0.88 - 0.43 * iroi, 0.15, 0.1])
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1.5,
        linecolor="royalblue",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=8,
        markeredgecolor='w',
    )
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        **depth_tuning_kwargs,
    )

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=16,
        which_speed="RS",
        speed_min=0,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd=1,
        markersize=3,
        linewidth=1,
        markeredgecolor='w',
        plot_x=0.25,
        plot_y=0.88 - 0.43 * iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=False,
    )
    ylim = plt.gca().get_ylim()

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd=1,
        markersize=3,
        linewidth=1,
        markeredgecolor='w',
        plot_x=0.44,
        plot_y=0.88 - 0.43 * iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=True,
        ylim=ylim,
    )
    plt.gca().set_ylabel("")

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=0,
        vmax=1,
        xlabel="Running speed (cm/s)",
        ylabel="Optic flow speed (degrees/s)",
        plot_x=0.04,
        plot_y=0.67 - 0.43 * iroi,
        plot_width=0.13,
        plot_height=0.13,
        cbar_width=0.01,
        fontsize_dict=fontsize_dict,
    )

    # 1 example neuron, fits of 5 models
    for imodel, (model, model_label) in enumerate(zip(models, model_labels)):
        if imodel == 0:
            ylabel = "Optic flow speed (degrees/s)"
        else:
            ylabel = ""
        if imodel == 1:
            xlabel = "Running speed (cm/s)"
        else:
            xlabel = ""

        closed_loop_rsof.plot_RS_OF_fit(
            fig=fig,
            neurons_df=neurons_df_example,
            roi=roi,
            model=model,
            model_label=model_label,
            min_sigma=0.25,
            vmin=0,
            vmax=1,
            log_range={
                "rs_bin_log_min": 0,
                "rs_bin_log_max": 2.5,
                "rs_bin_num": 6,
                "of_bin_log_min": -1.5,
                "of_bin_log_max": 3.5,
                "of_bin_num": 11,
                "log_base": 10,
            },
            plot_x=0.24 + 0.08 * imodel,
            plot_y=0.67 - 0.43 * iroi,
            plot_width=0.1,
            plot_height=0.1,
            xlabel=xlabel,
            ylabel=ylabel,
            fontsize_dict=fontsize_dict,
            cbar_width=None,
        )
        if imodel > 0:
            plt.gca().set_yticks([])

# filter depth-selective neurons with its preferred depth amplitude
neurons_df_sig = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["preferred_depth_amplitude"] > 0.2)
]
print(f"model comparison: Number of neurons: {len(neurons_df_sig)}, sessions: {len(neurons_df_sig.session.unique())}")
# Compare R2 of different models
_ = closed_loop_rsof.plot_r2_comparison(
    fig=fig,
    neurons_df=neurons_df_sig,
    models=models,
    labels=model_labels,
    plot_type="bar",
    markersize=3,
    alpha=0.7,
    color="k",
    plot_x=0.73, #0.75
    plot_y=0.76, #0.72
    plot_width=0.25,
    plot_height=0.2,
    fontsize_dict=fontsize_dict,
)
plt.ylim([-0.02,1.02])

# Plot RS/OF ratio - preferred depth scatter plot
results_all["RS_OF_ratio_closedloop_crossval_g2d"] = results_all[
    "preferred_RS_closedloop_crossval_g2d"
] / np.radians(results_all["preferred_OF_closedloop_crossval_g2d"])


neurons_df = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["rsof_test_rsq_closedloop_g2d"] > 0.02) 
]
print(f"RSOF ratio: Number of neurons: {len(neurons_df)}, sessions: {len(neurons_df.session.unique())}")

scatter_props = {
    "s": 0.5,
    "alpha": 0.2,
    "plot_diagonal": False,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}

r, p = closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df,
    xcol="RS_OF_ratio_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Ratio of preferred running \nand optic flow speeds (cm)",
    ylabel="Preferred virtual \ndepth (cm)",
    plot_x=0.07,
    plot_y=0.48,
    plot_width=0.21,
    plot_height=0.2,
    color="royalblue",  
    contour_color="k",
    linewidth=1,
    **scatter_props,
)

# Scatterplot of preferred RS and OF speeds (cross-validated) colored by preferred depth
closed_loop_rsof.plot_speed_colored_by_depth(
    fig=fig,
    neurons_df=neurons_df,
    xcol="preferred_RS_closedloop_crossval_g2d",
    ycol="preferred_OF_closedloop_crossval_g2d",
    zcol="preferred_depth_closedloop_crossval",
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred optic flow speed (degrees/s)",
    zlabel="Preferred virtual \ndepth (cm)",
    s=3,
    alpha=1,
    cmap="cool_r",
    plot_x=0.07,
    plot_y=0.2,
    plot_width=0.16,
    plot_height=0.35,
    fontsize_dict=fontsize_dict,
    edgecolors="w",
)

# Plot open-closedloop RS / OF trace
positions=[]
x0, y0, width, height = 0.35, 0.48, 0.13, 0.03
for i in range(4):
    positions.append([x0 + i//2 * width*1.1, y0 - i%2*height*1.3, width, height])
closed_loop_rsof.plot_speed_trace_closed_open_loop(flexilims_session=flexilims_session, 
                                  session_name=session_name_openloop_trace, 
                                  trials_df=trials_df_example_openloop_trace, 
                                  trial_list = [25,27,32], 
                                  positions=positions,
                                  linewidth=0.5,
                                  fontsize_dict={"title": 7, "label": 5, "tick": 5, "legend": 5})

# Plot example cell RS OF matrix (closed vs open loop)
EXAMPLE_ROIS_OPENLOOP = [857]
for iroi, roi in enumerate(EXAMPLE_ROIS_OPENLOOP):
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) - 1):
        xlabel = ""
    else:
        xlabel = "Running speed (cm/s)"
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) // 2):
        ylabel = ""
    else:
        ylabel = "Optic flow speed (degrees/s)"
    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=0,
        vmax=None,
        xlabel="",
        ylabel=ylabel,
        title="Closed loop",
        plot_x=0.66,
        plot_y=0.47 - 0.3 * iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=None,
        fontsize_dict=fontsize_dict,
    )

    closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=0,
        vmin=0,
        vmax=vmax,
        xlabel=xlabel,
        ylabel="",
        title="Open loop",
        plot_x=0.83,
        plot_y=0.47 - 0.3 * iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.005,
        fontsize_dict=fontsize_dict,
    )

# Plot preferred RS & OF closedloop vs openloop
results_all_openloop["amplitude_closedloop"] = results_all_openloop["rsof_popt_closedloop_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
results_all_openloop["amplitude_openloop"] = results_all_openloop["rsof_popt_openloop_actual_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
neurons_df_sig_openloop = results_all_openloop[
    (results_all_openloop["iscell"] == 1)
    & (results_all_openloop["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all_openloop["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all_openloop["rsof_test_rsq_closedloop_g2d"] > 0.02) 
    & (results_all_openloop["rsof_test_rsq_openloop_actual_g2d"] > 0.02)
]
print(f"open loop scatter plots: N neurons {len(neurons_df_sig_openloop)}, sessions {len(neurons_df_sig_openloop.session.unique())}")

scatter_props = {
    "s": 2,
    "alpha": 0.3,
    "plot_diagonal": True,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}
closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig_openloop,
    xcol="preferred_RS_closedloop_g2d",
    ycol="preferred_RS_openloop_actual_g2d",
    xlabel="Preferred running speed \nin closed loop (cm/s)",
    ylabel="Preferred running speed \nin open loop (cm/s)",
    color="royalblue",
    contour_color="k",
    plot_x=0.37,
    plot_y=0.28,
    plot_width=0.12,
    plot_height=0.12,
    **scatter_props,
)

closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig_openloop,
    xcol="preferred_OF_closedloop_g2d",
    ycol="preferred_OF_openloop_actual_g2d",
    xlabel="Preferred optic flow speed \nin closed loop (degrees/s)",
    ylabel="Preferred optic flow speed \nin open loop (degrees/s)",
    color="royalblue",
    contour_color="k",
    plot_x=0.6,
    plot_y=0.28,
    plot_width=0.12,
    plot_height=0.12,
    **scatter_props,
)

# Plot histogram of closedloop / openloop amplitude
ax = fig.add_axes([0.82, 0.29, 0.16, 0.1])
ratios = neurons_df_sig_openloop["amplitude_closedloop"] / neurons_df_sig_openloop["amplitude_openloop"]
print(f"closed loop amp / open loop amp median: {np.median(ratios)}")
print(f"1-sample wilcoxon test: {stats.wilcoxon(ratios-1)}")
bins = np.geomspace(1e-1, 1e1, 21)
n,_,_ = ax.hist(ratios, bins=bins, 
        color="cornflowerblue",
        edgecolor="royalblue",
        alpha=1)
ax.set_xscale("log")
ax.tick_params(axis='both', which='major', labelsize=fontsize_dict["tick"])
ax.set_xlabel("Peak response closedloop \n/ peak response open loop", fontsize=fontsize_dict["label"])
ax.set_ylabel("Number of neurons", fontsize=fontsize_dict["label"])
ax.axvline(1, color="k", linestyle="dotted", linewidth=1)
# ax.set_ylim([0, (np.nanmax(n)//500+(np.bool(np.nanmax(n)%500))+1)*500])
ax.set_ylim([0, 300])
ax.plot(
    np.median(ratios),
    ax.get_ylim()[1]*1,
    marker="v",
    markersize=5,
    markerfacecolor="cornflowerblue",
    markeredgecolor="royalblue",
)
ax.set_xlim([1e-1, 1e1])
sns.despine(ax=ax, offset=3, trim=True)

# Plot decoding results 
fig.add_axes([0.15, 0.03, 0.18, 0.18])
# Plot accuracy
depth_decoder.decoder_accuracy(
    decoder_results,
    plot_type="scatter",
    markersize=10,
    colors=["g", "royalblue"],
    markers=["o", "o"],
    markerfacecolors=["k", "none"],
    linewidth=1,
    xlabel=["Closed loop", "Open loop"],
    ylabel="Decoding accuracy",
    xlim=[0,0.8],
    ylim=[0,0.8],
    fontsize_dict=fontsize_dict,
) 

# Plot conmat
ndepths = 8
conmat_mean = depth_decoder.calculate_average_confusion_matrix(
    decoder_results[decoder_results["ndepths"] == ndepths]
)

ax1=fig.add_axes([0.42, 0.03, 0.18, 0.18])
ax2=fig.add_axes([0.7, 0.03, 0.18, 0.18])
depth_decoder.plot_closed_open_conmat(
    conmat_mean,
    normalize=True,
    fig=fig,
    ax1=ax1,
    ax2=ax2,
    fontsize_dict=fontsize_dict,
    vmax=0.45,
)
plt.savefig(SAVE_ROOT / "fig2.pdf", bbox_inches="tight", dpi=300)

## STATS


In [ ]:
# closedloop ratio vs preferred depth
n_boots = 20000
xcol="RS_OF_ratio_closedloop_crossval_g2d"
ycol="preferred_depth_closedloop_crossval"
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(neurons_df, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
pval = v1_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
print(f"closedloop ratio vs preferred depth: r={r}, p={pval}")

In [ ]:
# closed loop vs open loop preferred RS & preferred OF correlation stats
n_boots = 20000
xcol = "preferred_RS_closedloop_g2d"
ycol = "preferred_RS_openloop_actual_g2d"
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(neurons_df_sig_openloop, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
pval = v1_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
print(f"preferred RS closed loop vs open loop correlation: {r}, pval: {pval}")

xcol = "preferred_OF_closedloop_g2d"
ycol = "preferred_OF_openloop_actual_g2d"
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(neurons_df_sig_openloop, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
pval = v1_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
print(f"preferred OF closed loop vs open loop correlation: {r}, pval: {pval}")

In [ ]:
# closed loop vs open loop RS: are they significantly different?
n_boots = 20000
xcol = "preferred_RS_closedloop_g2d"
ycol = "preferred_RS_openloop_actual_g2d"
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(neurons_df_sig_openloop, 
                                               n_boots, 
                                               xcol, 
                                               ["mouse", "session"], 
                                               ycol=ycol, 
                                               correlation=False,
                                               difference=False,
                                               ratio=True)
pval = v1_common_utils.calculate_pval_from_bootstrap(distribution, value=1)
print(f"preferred RS closed loop vs open loop bigger? median{r}, pval: {pval}")

In [ ]:
# closed loop vs open loop RS: are they significantly different?
n_boots = 20000
xcol = "amplitude_closedloop"
ycol = "amplitude_openloop"
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(neurons_df_sig_openloop, 
                                               n_boots, 
                                               xcol, 
                                               ["mouse", "session"], 
                                               ycol=ycol, 
                                               correlation=False,
                                               difference=False,
                                               ratio=True)
pval = v1_common_utils.calculate_pval_from_bootstrap(distribution, value=1)
print(f"preferred RS closed loop vs open loop bigger? median{r}, pval: {pval}")

In [ ]:
# model comparison
model_cols = [f"rsof_test_rsq_closedloop_{model}" for model in models]
neurons_df_sig["best_model"] = neurons_df_sig[model_cols].idxmax(axis=1)
neuron_sum = (
    neurons_df_sig.groupby("session")[["roi"]].agg(["count"]).values.flatten()
)
props = []
# calculate the proportion of neurons that have the best model for each session
for i, model in enumerate(model_cols):
    prop = (
        neurons_df_sig.groupby("session")
        .apply(lambda x: x[x["best_model"] == model][["roi"]].agg(["count"]))
        .values.flatten()
    ) / neuron_sum
    props.append(prop)
props = np.vstack(props)

sessions_df = pd.DataFrame(props.T, columns=model_labels)
sessions_df["session"] = neurons_df_sig.session.unique()

n_boots = 20000
np.random.seed(0)
r, distribution = v1_common_utils.hierarchical_bootstrap_stats(sessions_df, 
                                               n_boots, 
                                               xcol=model_labels, 
                                               resample_cols=["mouse"], 
                                               ycol=None, 
                                               correlation=False,
                                               difference=False,
                                               ratio=False)
plt.figure()
for i in range(5):
    plt.subplot(3,2,i+1)
    pval = v1_common_utils.calculate_pval_from_bootstrap(distribution[:,3] - distribution[:,i], value=0)
    print(f"{model_labels[i]} vs {model_labels[3]}: median {np.median(distribution[:,3] - distribution[:,i])}, pval: {pval}")
    plt.hist(distribution[:,i] - distribution[:,3], bins=21)
    plt.axvline(np.percentile(distribution[:,3] - distribution[:,i], 2.5), color="r", linestyle="--")
    plt.axvline(np.percentile(distribution[:,3] - distribution[:,i], 97.5), color="r", linestyle="--")
    plt.axvline(0, color="k", linestyle="--")